In [2]:
using Revise
using FFTW, Optim, Images, ImageView, TestImages
using Noise
using DeconvOptim
using Statistics
using BenchmarkTools
using Deconvolution
using Zygote
using Tullio
using Plots
#BenchmarkTools.DEFAULT_PARAMETERS.samples = 10
#BenchmarkTools.DEFAULT_PARAMETERS.seconds = 100

┌ Info: Precompiling DeconvOptim [03e7cd2f-1a03-4ea9-b59b-760a446df67f]
└ @ Base loading.jl:1278


## Load the donut data and estimate PSF for it

In [2]:
img = convert(Array{Float32}, load("../dataset/real-donut/data.tif"));

psf = convert(Array{Float32}, load("../dataset/real-donut/estimated-psf.tif"));
psf_n = center_extract(psf, size(img)[1:2]);
img_n = view(img, :, :, 31:80);
size(psf_n) == size(img_n)

img_n2 = view(img_n, :, :, 1:5)
i1, i2 = get_indices_around_center(50, 5)
psf_n2 = ifftshift(psf_n[:, :, i1:i2]) 

┌ Info: Precompiling ImageMagick [6218d12a-5da1-5696-b52f-db25d2ecc6d1]
└ @ Base loading.jl:1278
┌ Warning: some versions of ImageMagick give spurious low-order bits for 32-bit TIFFs
└ @ ImageMagick /home/fxw/.julia/packages/ImageMagick/0LwpT/src/ImageMagick.jl:100


600×600×5 Array{Float32,3}:
[:, :, 1] =
 0.00181582   0.0012665    …  1.5259f-5  0.000366217  0.0012665
 0.0012665    0.000854505     0.0        0.000213626  0.000854505
 0.000350958  0.000213626     1.5259f-5  3.0518f-5    0.000213626
 0.0          0.0             3.0518f-5  1.5259f-5    0.0
 3.0518f-5    3.0518f-5       1.5259f-5  3.0518f-5    3.0518f-5
 1.5259f-5    1.5259f-5    …  0.0        0.0          1.5259f-5
 0.0          0.0             0.0        0.0          0.0
 0.0          0.0             0.0        0.0          0.0
 0.0          0.0             0.0        0.0          0.0
 0.0          0.0             0.0        0.0          0.0
 0.0          0.0          …  0.0        0.0          0.0
 0.0          0.0             0.0        0.0          0.0
 0.0          0.0             0.0        0.0          0.0
 ⋮                         ⋱                          
 0.0          0.0             0.0        0.0          0.0
 0.0          0.0             0.0        0.0          0.0
 

In [33]:
img = convert(Array{Float32}, load("../dataset/obj.tif"));

r = 30
psf = convert(Array{Float32}, DeconvOptim.generate_psf(img, r))

#psf = ifftshift(convert(Array{Float32}, load("../dataset/psf.tif")));
psf = psf ./ sum(psf)
otf = fft(psf)
img_b = max.(0, DeconvOptim.conv_otf(img, otf))
img_noisy = poisson(img_b, 100)
img_noisy ./= maximum(img_noisy)

imshow(abs.(img_b))

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>465: "map(clim-mapped image, inpu…
  "annotations" => 414: "input-194" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 413: "CLim" = CLim{Float32}(0.0001087, 0.00033859) CLim{Floa…

In [67]:
reg = DeconvOptim.TV(sum_dims=[1, 2], weights=[1, 1], λ=0.01, mode="central", step=1)
@time res, o = deconvolution(img_noisy[:,:,1], psf[:,:,1], 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=reg,
        options=Optim.Options(iterations=10), plan_fft=true, padding=true)

lollol  0.935219 seconds (2.20 M allocations: 103.604 MiB, 2.74% gc time)


(Float32[NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN],  * Status: failure (line search failed)

 * Candidate solution
    Final objective value:     2.215986e-03

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    1
    ∇f(x) calls:   1
)

In [35]:
imshow(res_nopad[:, :, :, 1, 1])
imshow(res[:, :, :, 1, 1])
#imshow(img_noisy)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>410: "map(clim-mapped image, inpu…
  "annotations" => 359: "input-232" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 358: "CLim" = CLim{Float32}(2.88292f-14, 0.0103583) CLim{Flo…

In [52]:
imshow(psf)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>55: "map(clim-mapped image, input…
  "annotations" => 4: "input-3" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 3: "CLim" = CLim{Float32}(0.0, 0.00348909) CLim{Float32} 

In [18]:
x = randn((12,14223, 1, 1));
x == center_extract(x, size(x))

true